In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Reshape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
of = pd.read_csv("/kaggle/input/combined-dataset-before-smote/output-2.csv", delimiter=',', header=None, dtype=float)
df = pd.read_csv("/kaggle/input/synthetic-dataset-with-pca/Synthetic_Dataset_with_PCA.csv", delimiter=',', header=None, dtype=float)
# Convert DataFrame to numpy array
synthetic = df.values
print(synthetic.shape)
outputs = of.values
# Separate features (inputs) and labels (outputs)
X = synthetic[1:, :]  # Features are all columns except the last 4
y = outputs[:, -4:]  # Label is the last column

# Split into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Ensure the dataset is not empty and has the expected dimensions
if synthetic.ndim != 2:
    raise ValueError("Dataset loaded is not in the expected 2D format.")

# Define the RNN model
model = Sequential([
    Reshape((x_train.shape[1], 1), input_shape=(x_train.shape[1],)),  # Reshape to (sequence_length, 1)
    SimpleRNN(11),  # RNN layer with 10 units
    Dense(1)  # Output layer with 4 units for 4 outputs
])


# Compile the model
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss = model.evaluate(x_test, y_test)
print("Test loss:", loss)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn import metrics

y_pred = model.predict(x_test)


# Flatten the predicted and actual values
y_pred_flat = y_pred.flatten()
y_test_flat = y_test.flatten()

# Convert to binary classification (example)
# Adjust this step according to your specific problem
threshold = 0.5
y_pred_binary = (y_pred_flat > threshold).astype(int)

# Reshape y_test_flat to match the shape of y_pred_flat
y_test_binary = (y_test_flat[:len(y_pred_binary)] > threshold).astype(int)
#y_pred = to_categorical(y_pred)
f1 = f1_score(y_test_binary, y_pred_binary, average='weighted')
# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)

cm = confusion_matrix(y_test_binary.ravel(), y_test_binary.ravel())
#np.savetxt('cm.txt', cm, delimiter=',', fmt='%f')
#f1 = f1_score(y_test.ravel(), y_pred.ravel())
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("TPR Score:", tpr_value)
print("FPR Score:", fpr_value)


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

precision_a, recall_a, threshold = precision_recall_curve(y_test_binary, y_pred_binary)
prc_auc = auc(recall_a, precision_a)
print("Area Under the PR Curve score: ", prc_auc)


fpr, tpr, threshold = roc_curve(y_test_binary, y_pred_binary)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(prc_auc))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision_a.tolist()
recall_list = recall_a.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": f1, 
    "Average Precision": precision, 
    "Average Recall": recall, 
    "Average True Positive Rate": tpr_value, 
    "Average False Positive Rate": fpr_value, 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": prc_auc,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/RNN - Synthetic Dataset with PCA.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)